In [1]:
#Import necessary packages
from google.cloud import bigquery
from google.oauth2 import service_account
import random
import sqlite3

In [2]:
#Set environmental varaible to get credentials
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/jackc/OneDrive/Documents/MSBA/Applied Data Analytics/Wedge/cloughwedge2021-3249989ff592.json"

In [3]:
#Connect Pyhton to my GBQ project
client = bigquery.Client(project = 'cloughwedge2021')
gbq_proj_id = "cloughwedge2021"

In [4]:
#Sales by date by hour

#Creating the first query. The goal is to query sales per hour for each calendar day. Additionally, we want
#the amount sold in each hour of each day, as well as the amount of transactions and amount of items sold.
#In the line where we get the amount of transactions, we want each transaction for distinct dates, register numbers,
#employee numbers, and transaction numbers all together.
sales_date_hour = '''SELECT (EXTRACT(DATE FROM DateTime)) AS Date,
    (EXTRACT(HOUR FROM DateTime)) AS Hour,
    ROUND(SUM(Total),2) as Sales,
    COUNT(DISTINCT(Date(DateTime) || Register_Num || Emp_Num || Trans_Num)) AS Transactions,
    SUM(CASE WHEN(Trans_Status = 'V' OR Trans_Status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `cloughwedge2021.cloughwedge2021.transArchive*`
    
    WHERE Card_No != 3
    AND Department != 0
    AND Department != 15
    AND Trans_Status != 'M'
    AND Trans_Status != 'C'
    AND Trans_Status != 'J'
    AND (Trans_Status = ''
    OR Trans_Status = ' '
    OR Trans_Status = 'V'
    OR Trans_Status = 'R')
    GROUP BY Date, Hour
    ORDER BY Date, Hour'''
date_by_hour = client.query(sales_date_hour,location="US")

In [5]:
#Define the headers, giving each queried variable a place to go
headers = ["Date","Hour","Sales","Transactions","Items"]

In [6]:
#Create a file to write our results to. It will iterate over each row in the query results and write them to the file
with open("sales_date_hour.txt", 'w', encoding = 'utf-8') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in date_by_hour:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [7]:
#Create and connect to our database in SQLite.
#A cursor is "an opaque base64-encoded string marking the index position of the last result retrieved", so basically
#a way of indexing a database/query
db = sqlite3.connect("WedgeTask3.db")
c = db.cursor()

In [8]:
input_file = "sales_date_hour.txt"

#Here we create our sales by day by hour. The cursor object is required to do any of these things.
#We start with dropping a table if it already exists, as you do in SQL. We create a table with the variables we need
#and then write them to the file we created and wrote to earlier. 
c.execute('''DROP TABLE IF EXISTS Daily_Hourly_Sales''')

c.execute('''CREATE TABLE Daily_Hourly_Sales (
    Date TIMESTAMP,
    Hour TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')

#There are 5 comma separated values in each line of our query results. This loop will add each of those values
#to the appropriate position in our text file. Once it reaches the end of a line, it will go to the next line and
#add each value to the appropriate spot again (dates are the first index position, hour is the second, etc.)
with open(input_file, 'r', encoding = 'utf-8') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        c.execute('''
        INSERT INTO Daily_Hourly_Sales (Date, Hour, Sales, Transactions, Items)
        VALUES (?,?,?,?,?)''', line)
        

#This will commit the text file to a database table       
db.commit()

In [9]:
#Sales by owner by year by month

#Create a table that exludes certain values with !=. Group By will put all of our results together by 
#owner ID, the year of the transaction, and the month. Thus, we get sales by owner by year by month.
#We also want to exclude values listed with a transaction status of member dsicounts, coupons, or juice club cards
#because those are not transactions, they're just details of disounts. That ensures we are only left with
#typical value transactions, voids, or returns.

query_owner_year_month = '''SELECT Card_No as Owner_ID,
    (EXTRACT(Year FROM DateTime)) AS Year,
    (EXTRACT(Month FROM DateTime)) AS Month,
    ROUND(SUM(Total),2) as Sales,
    COUNT(DISTINCT(Date(DateTime) || Register_Num || Emp_Num || Trans_Num)) AS Transactions,
    SUM(CASE WHEN(Trans_Status = 'V' OR Trans_Status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `cloughwedge2021.cloughwedge2021.transArchive*`

    WHERE Department != 0
    AND Department != 15
    AND Trans_Status != 'M'
    AND Trans_Status != 'C'
    AND Trans_Status != 'J'
    AND (Trans_Status = ''
    OR Trans_Status = ' '
    OR Trans_Status = 'V'
    OR Trans_Status = 'R')
    GROUP BY Card_No, Year, Month
    ORDER BY Card_No, Year, Month'''

owner_year_month = client.query(query_owner_year_month,location="US")

In [10]:
headers = ["Owner_ID","Year","Month","Sales","Transactions","Items"]

In [11]:
with open("sales_owner_year_month.txt", 'w', encoding = 'utf-8') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in owner_year_month:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [12]:
input_file2 = "sales_owner_year_month.txt"

c.execute('''DROP TABLE IF EXISTS yearly_monthly_sales_by_owner''')

c.execute('''CREATE TABLE yearly_monthly_sales_by_owner (
    Owner_ID INTEGER,
    Year TIMESTAMP,
    Month TIMESTAMP,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')

with open(input_file2, 'r', encoding = 'utf-8') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        c.execute('''
        INSERT INTO yearly_monthly_sales_by_owner (Owner_ID, Year, Month, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?)''', line)
        
db.commit()

In [13]:
#Sales by product description by year by month

prod_desc_year_month_query = '''SELECT Department,
    (EXTRACT(Year FROM DateTime)) AS Year,
    (EXTRACT(Month FROM DateTime)) AS Month,
    UPC,
    Description,
    ROUND(SUM(Total),2) as Sales,
    COUNT(DISTINCT(DATE(DateTime) || Register_Num || Emp_Num || Trans_Num)) AS Transactions,
    SUM(CASE WHEN(Trans_Status = 'V' OR Trans_Status = 'R') THEN -1 ELSE 1 END) as Items
    FROM `cloughwedge2021.cloughwedge2021.transArchive*`
    
    WHERE Card_No != 3
    AND Department != 0
    AND Department != 15
    AND Trans_Status != 'M'
    AND Trans_Status != 'C'
    AND Trans_Status != 'J'
    AND (Trans_Status = ''
    OR Trans_Status = ' '
    OR Trans_Status = 'V'
    OR Trans_Status = 'R')
    GROUP BY Year, Month, UPC, Description, Department
    ORDER BY Description, Year, Month DESC'''    
    
product_description_year_month = client.query(prod_desc_year_month_query,location="US")

In [14]:
headers = ["Department","Year","Month","UPC","Sales","Description","Transactions","Items"]

In [15]:
with open("sales_product_desc_year_month.txt", 'w', encoding = 'utf-8') as outfile:
    outfile.write(",".join(headers) + "\n")
    for row in product_description_year_month:
         outfile.write(",".join([str(item) for item in row])+ '\n')

In [16]:
input_file3 = "sales_product_desc_year_month.txt"

c.execute('''DROP TABLE IF EXISTS prod_description_by_year_by_month''')

c.execute('''CREATE TABLE prod_description_by_year_by_month (
    Department FLOAT,
    Dept_name TEXT,
    Year TIMESTAMP,
    Month TIMESTAMP,
    UPC STRING,
    Description STRING,
    Sales REAL,
    Transactions INTEGER,
    Items INTEGER)''')

with open(input_file3, 'r', encoding = 'utf-8') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        c.execute('''
        INSERT INTO prod_description_by_year_by_month (Department, Year, Month, UPC, Description, Sales, Transactions, Items)
        VALUES (?,?,?,?,?,?,?,?)''', line)
        
db.commit()